[View in Colaboratory](https://colab.research.google.com/github/jdelano18/Flight-Predictor/blob/master/flight_predictor.ipynb)

#Flight Predictor Project -- 94.1% accuracy

---

###By: Jimmy DeLano  [https://github.com/jdelano18] <br> Created: 5/2018 -- Senior in high school at [Milton Academy](https://www.milton.edu/)



###1.   Introduction
###2.   Data Preperation 
> 2.1 Loading Data 
<br>
> 2.2 Cleaning data
<br>
> 2.3 Creating Weather-Based Features
<br>
> 2.4 Null and Missing Values



###3. Model Creation
> 3.1 Evaluating Baseline
<br>
> 3.2 Catagorical Values
<br>
> 3.3 Evaluating First Classification Models
<br>
> 3.4 Model Simplification/Feature Elimination
<br>
> 3.5 Other Unsuccessful Feature Engineering Techniques
<br>
> 3.6 Final Features and Model













##1. Indroduction

I used this data from kaggle https://www.kaggle.com/fabiendaniel/predicting-flight-delays-tutorial/data to create a 

In [0]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier

## 2. Data Preperation

###2.1 Loading Data

In [0]:
#google colabratory specific code -- authenticate access to google drive
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
#list files in parent folder
file_list = drive.ListFile({'q': "'1DXp7GGE2oYBXcBBIlAK2_dPSNN5f3Q_s' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: test-colab.ipynb, id: 1xlpO1TNpj4QuaMc1kCcoVh8xbrD30Af4
title: midway_model.csv, id: 1bbsNO7bkdyHDApDb9YPusx2cLxs3bdUE
title: airports.csv, id: 1dI3_hhvvA50s-Foxlwlk0ysOtuEb9LPI
title: airlines.csv, id: 1yTScC9dM9e1NpcWt9cIVCUeMQNcuqZKU
title: USC00111577.csv, id: 1NvrFLAsKf-FhkT3EFKdjWPCfVsCbf8Tn
title: flights.csv, id: 1KBYpVdtwIPDRdU2BHiQRD7dAGueljzPI


In [0]:
#load datasets
midway_model = drive.CreateFile({'id': '1bbsNO7bkdyHDApDb9YPusx2cLxs3bdUE'})
midway_model.GetContentFile('midway_model.csv')
airports = drive.CreateFile({'id': '1dI3_hhvvA50s-Foxlwlk0ysOtuEb9LPI'})
airports.GetContentFile('airports.csv')
airlines = drive.CreateFile({'id': '1yTScC9dM9e1NpcWt9cIVCUeMQNcuqZKU'})
airlines.GetContentFile('airlines.csv')
chicago = drive.CreateFile({'id': '1NvrFLAsKf-FhkT3EFKdjWPCfVsCbf8Tn'})
chicago.GetContentFile('USC00111577.csv')
flights = drive.CreateFile({'id': '1KBYpVdtwIPDRdU2BHiQRD7dAGueljzPI'})
flights.GetContentFile('flights.csv')

In [54]:
chicago = pd.read_csv('USC00111577.csv', low_memory = False)
print(chicago.shape)
chicago.head()

(24361, 131)


,StnID,Lat,Lon,Elev,Year-Month-Day,Element,HR00Val,HR00MF,HR00QF,HR00S1,...,HR23Val,HR23MF,HR23QF,HR23S1,HR23S2,DlySum,DlySumMF,DlySumQF,DlySumS1,DlySumS2
0,USC00111577,41.7372,-87.7775,189.0,1948-07-01,HPCP,0,g,,4,...,0,Z,,4,,0,,,,C
1,USC00111577,41.7372,-87.7775,189.0,1948-07-02,HPCP,0,Z,,4,...,0,Z,,4,,0,,,,C
2,USC00111577,41.7372,-87.7775,189.0,1948-07-03,HPCP,0,Z,,4,...,0,Z,,4,,0,,,,C
3,USC00111577,41.7372,-87.7775,189.0,1948-07-04,HPCP,0,Z,,4,...,0,Z,,4,,0,,,,C
4,USC00111577,41.7372,-87.7775,189.0,1948-07-05,HPCP,0,Z,,4,...,0,Z,,4,,0,,,,C


In [50]:
airlines = pd.read_csv('airlines.csv')
print(airlines.shape)
airlines.head()

(14, 2)


,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [51]:
airports = pd.read_csv('airports.csv')
print(airports.shape)
airports.head()

(322, 7)


,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [52]:
flights = pd.read_csv('flights.csv', low_memory = False)
print(flights.shape)
flights.head()

(5819079, 31)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


###2.2 Cleaning Data

In [10]:
#combining dataframes and dropping unnecessary columns
%%time
df = pd.merge(flights, airlines,left_on='AIRLINE', right_on='IATA_CODE')
df['AIRLINE_x'] = df['AIRLINE_y']
df = df.drop(columns=['IATA_CODE','AIRLINE_y'])
df.rename(columns={'AIRLINE_x' : 'AIRLINE'}, inplace = True)
df = pd.merge(df,airports,left_on='ORIGIN_AIRPORT',right_on='IATA_CODE')
df['ORIGIN_AIRPORT'] = df['AIRPORT']
df = df.drop(columns=['IATA_CODE','AIRPORT'])
df.rename(columns={'CITY': 'ORIGIN_CITY','STATE':'ORIGIN_STATE','COUNTRY':'ORIGIN_COUNTRY','LATITUDE':'ORIGIN_LATITUDE','LONGITUDE':'ORIGIN_LONGITUDE'}, inplace=True)
df = pd.merge(df,airports,left_on='DESTINATION_AIRPORT',right_on='IATA_CODE')
df['DESTINATION_AIRPORT'] = df['AIRPORT']
df = df.drop(columns=['IATA_CODE','AIRPORT'])
df.rename(columns={'CITY': 'DESTINATION_CITY','STATE':'DESTINATION_STATE','COUNTRY':'DESTINATION_COUNTRY','LATITUDE':'DESTINATION_LATITUDE','LONGITUDE':'DESTINATION_LONGITUDE'}, inplace=True)
df = df.drop(columns=['FLIGHT_NUMBER', 'TAIL_NUMBER'])

CPU times: user 47.8 s, sys: 523 ms, total: 48.3 s
Wall time: 48.4 s


In [53]:
midway = df.loc[df['ORIGIN_AIRPORT'] == 'Chicago Midway International Airport']
midway['Year-Month-Day'] = pd.to_datetime(df[['YEAR','MONTH','DAY']])
midway = midway.drop(columns=['ORIGIN_AIRPORT','ORIGIN_CITY', 'ORIGIN_STATE', 'ORIGIN_COUNTRY',
       'ORIGIN_LATITUDE', 'ORIGIN_LONGITUDE', 'DESTINATION_CITY','DESTINATION_STATE', 'DESTINATION_COUNTRY',
        'DESTINATION_LATITUDE','DESTINATION_LONGITUDE','YEAR','MONTH','DAY', 'WHEELS_ON', 'TAXI_IN', 
        'ARRIVAL_TIME', 'ELAPSED_TIME', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON'])
midway = midway.reset_index(drop=True)
midway.dtypes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



DAY_OF_WEEK                     int64
AIRLINE                        object
DESTINATION_AIRPORT            object
SCHEDULED_DEPARTURE             int64
DEPARTURE_TIME                float64
DEPARTURE_DELAY               float64
TAXI_OUT                      float64
WHEELS_OFF                    float64
SCHEDULED_TIME                float64
AIR_TIME                      float64
DISTANCE                        int64
SCHEDULED_ARRIVAL               int64
ARRIVAL_DELAY                 float64
AIR_SYSTEM_DELAY              float64
SECURITY_DELAY                float64
AIRLINE_DELAY                 float64
LATE_AIRCRAFT_DELAY           float64
WEATHER_DELAY                 float64
Year-Month-Day         datetime64[ns]
dtype: object

In [13]:
print(midway.shape)
midway.head()

(80886, 19)


,DAY_OF_WEEK,AIRLINE,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,Year-Month-Day
0,4,Southwest Airlines Co.,Seattle-Tacoma International Airport,855,854.0,-1.0,9.0,903.0,280.0,251.0,1733,1135,-13.0,NaN,NaN,NaN,NaN,NaN,2015-01-01
1,4,Southwest Airlines Co.,Seattle-Tacoma International Airport,1925,1948.0,23.0,20.0,2008.0,275.0,246.0,1733,2200,18.0,0.0,0.0,2.0,16.0,0.0,2015-01-01
2,5,Southwest Airlines Co.,Seattle-Tacoma International Airport,855,901.0,6.0,19.0,920.0,280.0,244.0,1733,1135,-5.0,NaN,NaN,NaN,NaN,NaN,2015-01-02
3,5,Southwest Airlines Co.,Seattle-Tacoma International Airport,1925,1947.0,22.0,11.0,1958.0,275.0,265.0,1733,2200,30.0,8.0,0.0,8.0,14.0,0.0,2015-01-02
4,6,Southwest Airlines Co.,Seattle-Tacoma International Airport,850,927.0,37.0,13.0,940.0,280.0,246.0,1733,1130,23.0,0.0,0.0,5.0,18.0,0.0,2015-01-03


In [15]:
chicago['Year'] = chicago['Year-Month-Day'].apply(lambda x: x[0:4])
chicago = chicago.loc[chicago['Year'] == '2015']
chicago = chicago.drop(columns=['StnID','Lat','Lon','Elev','Element','DlySumS1','DlySumS2','HR00QF','HR00S1','HR00S2',
                                'HR01QF','HR01S1', 'HR01S2','HR02QF','HR02S1','HR02S2','HR03QF','HR03S1','HR03S2',
                                'HR04QF','HR04S1','HR04S2','HR05QF','HR05S1','HR05S2','HR06QF','HR06S1','HR06S2',
                                'HR07QF','HR07S1','HR07S2','HR08QF','HR08S1','HR08S2','HR09QF','HR09S1','HR09S2',
                               'HR10QF','HR10S1','HR10S2','HR11QF','HR11S1','HR11S2','HR12QF','HR12S1','HR12S2',
                               'HR13QF','HR13S1','HR13S2','HR14QF','HR14S1','HR14S2','HR15QF','HR15S1','HR15S2',
                               'HR16QF','HR16S1','HR16S2','HR17QF','HR17S1','HR17S2','HR18QF','HR18S1','HR18S2',
                               'HR19QF','HR19S1','HR19S2','HR20QF','HR20S1','HR20S2','HR21QF','HR21S1','HR21S2',
                               'HR22QF','HR22S1','HR22S2','HR23QF','HR23S1','HR23S2','Year','DlySumMF','DlySumQF',
                               'HR00MF','HR01MF','HR02MF','HR03MF','HR04MF','HR05MF','HR06MF','HR07MF','HR08MF','HR09MF',
                               'HR10MF','HR11MF','HR12MF','HR13MF','HR14MF','HR15MF','HR16MF','HR17MF','HR18MF','HR19MF',
                               'HR20MF','HR21MF','HR22MF','HR23MF'])

chicago['Year-Month-Day'] = pd.to_datetime(chicago['Year-Month-Day'])
chicago = chicago.reset_index(drop=True)
chicago.head()

,Year-Month-Day,HR00Val,HR01Val,HR02Val,HR03Val,HR04Val,HR05Val,HR06Val,HR07Val,HR08Val,...,HR15Val,HR16Val,HR17Val,HR18Val,HR19Val,HR20Val,HR21Val,HR22Val,HR23Val,DlySum
0,2015-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-02,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,3
2,2015-01-03,0,0,3,2,2,3,3,2,2,...,1,0,0,0,0,0,1,1,2,63
3,2015-01-04,0,0,2,2,2,5,2,1,1,...,0,0,0,0,0,0,0,0,0,20
4,2015-01-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,7,3,3,19


###2.3 Creating Weather-Based Features

In [0]:
#I created this function to get the value of precipitation from the 'chicago' dataframe based off the time of any flight -- the other parameters help to
#get the values for the 'precip_1hr_before' and 'daily_sum' variables.

def round_time_down(time, hours_before, get_daily_totals):
    val = ""
    sum_vals = []
    if get_daily_totals == False:
        time = time - (100*hours_before)
        if time < 100:
            return "HR00Val"
        else:
            time -= 100

        if time < 1000:
            hold = str(time)[:1]
            val = "HR0"+hold+"Val"
        else:
            hold = str(time)[:2]
            val = "HR"+hold+"Val"   
        return val   
    
    if get_daily_totals == True:
        for i in range(0, int(time/100)):
            if i < 10:
                sum_vals.append("HR0"+str(i)+"Val")
            else:
                sum_vals.append("HR"+str(i)+"Val")
        return sum_vals

In [17]:
#testing to see if function works properly
sum = 0
for i in round_time_down(midway.iloc[4]['SCHEDULED_DEPARTURE'],0, True):
    sum = sum+chicago.iloc[2][i]
print(sum)

15


In [18]:
%%time
precip_0hr_before = []
precip_1_hr_before = []
daily_sum = []

#small_poc = midway[:100]

for midway_index, midway_row in midway.iterrows():
    for chicago_index, chicago_row in chicago.iterrows():
        if midway_row['Year-Month-Day'] == chicago_row['Year-Month-Day']:
            precip_0hr_before.append(chicago.iloc[chicago_index][round_time_down(midway_row['SCHEDULED_DEPARTURE'], 0, False)])
            precip_1_hr_before.append(chicago.iloc[chicago_index][round_time_down(midway_row['SCHEDULED_DEPARTURE'], 1, False)])
            sum = 0
            for i in round_time_down(midway.iloc[midway_index]['SCHEDULED_DEPARTURE'],0, True):
                sum = sum + chicago.iloc[chicago_index][i]
            daily_sum.append(sum)
            break
#print(daily_sum)
print(precip_0hr_before, precip_1_hr_before, daily_sum)

KeyboardInterrupt: ignored

In [0]:
s1 = pd.Series(precip_0hr_before)
s2 = pd.Series(precip_1_hr_before)
s3 = pd.Series(daily_sum)

In [0]:
midway['precip_0hr_before'] = s1.values
midway['precip_1hr_before'] = s2.values
midway['daily_sum'] = s3.values
midway.head()

In [0]:
midway['Year-Month-Day'].apply(lambda x: x.month).value_counts()

In [0]:
arr1 = []
arr2 = []
for i in midway['Year-Month-Day'].unique():
    hold = midway.loc[midway['Year-Month-Day'] == i]
    arr1.append(i)
    arr2.append(hold['DEPARTURE_DELAY'].mean())

In [0]:
# plotly.offline.init_notebook_mode(connected=True)
# trace = go.Scatter(
#     x = arr1,
#     y = arr2,
#     mode = 'lines'
# )
# data = [trace]
# layout = go.Layout(
#     title='Delays Leaving Chicago Midway from 2015',
#     xaxis=dict(
#         title='Day of Year'
#     ),
#     yaxis=dict(
#         title='Average Departure Delay (mins)'
#     )
# )
# fig = go.Figure(data=data, layout=layout)
# plotly.offline.iplot(fig, filename='delay')

###2.4 Null and Missing Values

In [0]:
midway_model = midway.copy()
fill = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
for f in fill:
    midway_model[f] = midway_model[f].fillna(0)

midway_model = midway_model.dropna(axis=0, how='any')
print(midway.shape)
print(midway_model.shape)
midway_model.head()

In [0]:
#save this dataframe for easier access during model creation phase -- this way you don't have to re-run the cell that created weather features that took
#~25 minutes to run
midway_model.to_csv('midway_model.csv', index = False)

##3. Model Creation

###3.1 Evaluating Baseline

In [0]:
midway_model = pd.read_csv('midway_model.csv')

In [0]:
midway_model['Arrival_15'] = midway_model['ARRIVAL_DELAY'].apply(lambda x: 1 if x>= 15.0 else 0)

In [0]:
midway_model['Arrival_15'].value_counts()

In [0]:
print(1-(17273/(61412+17273)))

~78.0% of flights arrived within 15 minutes of their scheduled arrival time. This value represents the percent accuracy for a model that would just guess that every flight will not arive late. We'll use this baseline to evaluate the model -- we're aiming to get a lot better than this.

###3.2 Catagorical Variables

In [0]:
airlines = {0: 'Alaska Airlines Inc.', 1: 'American Airlines Inc.', 2:'American Eagle Airlines Inc.', 
            3:'Atlantic Southeast Airlines', 4:'Delta Air Lines Inc.', 5:'Frontier Airlines Inc.',
            6:'Hawaiian Airlines Inc.', 7:'JetBlue Airways', 8:'Skywest Airlines Inc.', 9:'Southwest Airlines Co.',
            10:'Spirit Air Lines', 11:'US Airways Inc.', 12:'United Air Lines Inc.', 13:'Virgin America'}
midway_model['AIRLINE'] = midway_model['AIRLINE'].map(airlines)
midway_model['Year-Month-Day'] = pd.to_datetime(midway_model['Year-Month-Day'])
#print(midway_model['Year-Month-Day'][0].month)
midway_model['Month'] = midway_model['Year-Month-Day'].apply(lambda x: x.month)
midway_model['Day'] = midway_model['Year-Month-Day'].apply(lambda x: x.day)
midway_model = midway_model.drop(columns = ['Year-Month-Day', 'DEPARTURE_TIME','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME',
                                           'AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])


In [22]:
midway_model = pd.get_dummies(data=midway_model, columns=['AIRLINE'], drop_first=True)
midway_model = pd.get_dummies(data=midway_model, columns=['DESTINATION_AIRPORT'], drop_first=True)
midway_model.head()

,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,precip_0hr_before,precip_1hr_before,daily_sum,...,DESTINATION_AIRPORT_St. Louis International Airport at Lambert Field,DESTINATION_AIRPORT_Tampa International Airport,DESTINATION_AIRPORT_Theodore Francis Green State Airport,DESTINATION_AIRPORT_Trenton Mercer Airport,DESTINATION_AIRPORT_Tucson International Airport,DESTINATION_AIRPORT_Tulsa International Airport,DESTINATION_AIRPORT_Washington Dulles International Airport,DESTINATION_AIRPORT_Wichita Dwight D. Eisenhower National Airport (Wichita Mid-Continent Airport),DESTINATION_AIRPORT_Will Rogers World Airport,DESTINATION_AIRPORT_William P. Hobby Airport
0,4,855,-1.0,251.0,1733,1135,-13.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1925,23.0,246.0,1733,2200,18.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,855,6.0,244.0,1733,1135,-5.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,5,1925,22.0,265.0,1733,2200,30.0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,6,850,37.0,246.0,1733,1130,23.0,2,3,15,...,0,0,0,0,0,0,0,0,0,0


0    61412
1    17273
Name: Arrival_15, dtype: int64

0.7804791256275021


In [24]:
X_features = midway_model.drop(['ARRIVAL_DELAY', 'Arrival_15'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_features,midway_model['Arrival_15'],test_size=0.3,random_state=5)
classifier = DecisionTreeClassifier()
classifier = classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

print(confusion_matrix(y_test, predictions))
print("Accuracy: {0:.1f}%".format(accuracy_score(y_test, predictions) * 100))

[[17251  1168]
 [ 1033  4154]]
Accuracy: 90.7%


In [26]:
X_features = midway_model.drop(['ARRIVAL_DELAY', 'Arrival_15'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_features,midway_model['Arrival_15'],test_size=0.3,random_state=5)
classifier = RandomForestClassifier()
classifier = classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)


print(confusion_matrix(y_test, predictions))
print("Accuracy: {0:.1f}%".format(accuracy_score(y_test, predictions) * 100))

[[18029   390]
 [ 1241  3946]]
Accuracy: 93.1%


In [27]:
%%time
dtc_parameters = {
    'criterion':['gini','entropy'],
    'splitter':['best','random'],
    'max_depth':[4,5],
    'min_samples_split':[2],
    'max_features':[78,79,80]
}
#cv = number of cross validation folds
dtc_gs = GridSearchCV(DecisionTreeClassifier(), dtc_parameters, cv=5, verbose = 0)
dtc_gs.fit(X_features,midway_model['Arrival_15'])
print(dtc_gs.best_score_)
print(dtc_gs.best_params_)
print(dtc_gs.best_estimator_)

0.9272542415962381
{'criterion': 'gini', 'max_depth': 5, 'max_features': 80, 'min_samples_split': 2, 'splitter': 'best'}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=80, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
CPU times: user 27.8 s, sys: 10 ms, total: 27.8 s
Wall time: 27.8 s


In [30]:
models = []

models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RFC', RandomForestClassifier()))
models.append(('BAG', BaggingClassifier()))
models.append(('ADA', AdaBoostClassifier()))


for name, model in models:
    model.fit(X_train, y_train)
    # predict on X_holdout
    yhat_ho = model.predict(X_test)
    print(name, "Holdout Results")
    print(round((accuracy_score(y_test, yhat_ho)*100),1),"% Accuracy")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, yhat_ho))
    print(classification_report(y_test, yhat_ho))

LR Holdout Results
94.2 % Accuracy
Confusion Matrix:
[[18001   418]
 [  942  4245]]
             precision    recall  f1-score   support

          0       0.95      0.98      0.96     18419
          1       0.91      0.82      0.86      5187

avg / total       0.94      0.94      0.94     23606

KNN Holdout Results
91.3 % Accuracy
Confusion Matrix:
[[18097   322]
 [ 1737  3450]]
             precision    recall  f1-score   support

          0       0.91      0.98      0.95     18419
          1       0.91      0.67      0.77      5187

avg / total       0.91      0.91      0.91     23606

CART Holdout Results
90.8 % Accuracy
Confusion Matrix:
[[17291  1128]
 [ 1039  4148]]
             precision    recall  f1-score   support

          0       0.94      0.94      0.94     18419
          1       0.79      0.80      0.79      5187

avg / total       0.91      0.91      0.91     23606

RFC Holdout Results
93.0 % Accuracy
Confusion Matrix:
[[18040   379]
 [ 1266  3921]]
             pr

In [0]:
# %%time
# lr_parameters = {
#     'penalty':['l1','l2'],
#     'tol':[0.0001,0.0005,0.001],
#     'C':[0.1, 1.0, 10.0],
#     'fit_intercept':[True, False],
#     'class_weight': ['balanced', None],
#     'n_jobs': [-1]
# }
# #cv = number of cross validation folds
# lr_gs = GridSearchCV(LogisticRegression(), lr_parameters, cv=5, verbose = 0)
# lr_gs.fit(X_features,midway_model['Arrival_15'])
# print(lr_gs.best_score_)
# print(lr_gs.best_params_)
# print(lr_gs.best_estimator_)

In [32]:
models = []
# recursive feature elimination
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
models.append(('RFE_LR', RFE(LogisticRegression(), n_features_to_select = 60)))
models.append(('RFE_RFC', RFE(RandomForestClassifier(), n_features_to_select = 60)))

for name, model in models:
    model.fit(X_train, y_train)
    yhat_ho = model.predict(X_test)

    print(name, "Holdout Results")
    print(round((accuracy_score(y_test, yhat_ho)*100),1),"% Accuracy")
    print(round((f1_score(y_test, yhat_ho)*100),0),"% f1-score")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, yhat_ho))
    print(classification_report(y_test, yhat_ho))
    print(model.support_.shape)
    
    col_drop = midway_model.columns
    top_attributes = []
    for i in range(model.support_.shape[0]):
        if model.support_[i] == True:
            top_attributes.append(col_drop[i])
            
    print("Here is a list of the most important features:")
    print(top_attributes)
    print("\n")

RFE_LR Holdout Results
92.8 % Accuracy
82.0 % f1-score
Confusion Matrix:
[[17928   491]
 [ 1211  3976]]
             precision    recall  f1-score   support

          0       0.94      0.97      0.95     18419
          1       0.89      0.77      0.82      5187

avg / total       0.93      0.93      0.93     23606

(83,)
Here is a list of the most important features:
['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'Day', 'AIRLINE_4', 'AIRLINE_5', 'AIRLINE_8', 'AIRLINE_9', 'DESTINATION_AIRPORT_Albuquerque International Sunport', 'DESTINATION_AIRPORT_Austin-Bergstrom International Airport', 'DESTINATION_AIRPORT_Bill and Hillary Clinton National Airport\xa0(Adams Field)', 'DESTINATION_AIRPORT_Birmingham-Shuttlesworth International Airport', 'DESTINATION_AIRPORT_Boise Airport\xa0(Boise Air Terminal)', 'DESTINATION_AIRPORT_Bradley International Airport', 'DESTINATION_AIRPORT_Buffalo Niagara International Airport', 'DESTINATION_AIRPORT_Cleveland Hopkins International Airport', 'DESTINATION_AIRPORT_Da

In [33]:
midway_model = pd.read_csv('midway_model.csv')
#print(midway.columns)
airlines = {0: 'Alaska Airlines Inc.', 1: 'American Airlines Inc.', 2:'American Eagle Airlines Inc.', 
            3:'Atlantic Southeast Airlines', 4:'Delta Air Lines Inc.', 5:'Frontier Airlines Inc.',
            6:'Hawaiian Airlines Inc.', 7:'JetBlue Airways', 8:'Skywest Airlines Inc.', 9:'Southwest Airlines Co.',
            10:'Spirit Air Lines', 11:'US Airways Inc.', 12:'United Air Lines Inc.', 13:'Virgin America'}
midway_model['AIRLINE'] = midway_model['AIRLINE'].map(airlines)
midway_model['Year-Month-Day'] = pd.to_datetime(midway_model['Year-Month-Day'])
#print(midway_model['Year-Month-Day'][0].month)
midway_model['Month'] = midway_model['Year-Month-Day'].apply(lambda x: x.month)
midway_model['Day'] = midway_model['Year-Month-Day'].apply(lambda x: x.day)
midway_model = midway_model.drop(columns = ['Year-Month-Day', 'DEPARTURE_TIME','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME',
                                           'AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
midway_model['Arrival_15'] = midway_model['ARRIVAL_DELAY'].apply(lambda x: 1 if x>= 15.0 else 0)
midway_model.head()

,DAY_OF_WEEK,AIRLINE,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,precip_0hr_before,precip_1hr_before,daily_sum,Month,Day,Arrival_15
0,4,Southwest Airlines Co.,Seattle-Tacoma International Airport,855,-1.0,251.0,1733,1135,-13.0,0,0,0,1,1,0
1,4,Southwest Airlines Co.,Seattle-Tacoma International Airport,1925,23.0,246.0,1733,2200,18.0,0,0,0,1,1,1
2,5,Southwest Airlines Co.,Seattle-Tacoma International Airport,855,6.0,244.0,1733,1135,-5.0,0,0,1,1,2,0
3,5,Southwest Airlines Co.,Seattle-Tacoma International Airport,1925,22.0,265.0,1733,2200,30.0,0,0,2,1,2,1
4,6,Southwest Airlines Co.,Seattle-Tacoma International Airport,850,37.0,246.0,1733,1130,23.0,2,3,15,1,3,1


In [34]:
airport_pivot = midway_model.pivot_table(values = 'Arrival_15', index = 'DESTINATION_AIRPORT')
airport_pivot.sort_values(by =['Arrival_15'], ascending = False).describe()

,Arrival_15
count,69.000000
mean,0.228113
std,0.069002
min,0.000000
25%,0.202152
50%,0.224147
75%,0.243217
max,0.568182


In [35]:
airport_pivot = airport_pivot.sort_values(by =['Arrival_15'], ascending = False)
top_5 = list(airport_pivot[:5].index)
top_25 = list(airport_pivot[5:17].index)
second_25 = list(airport_pivot[17:35].index)
third_25 = list(airport_pivot[35:53].index)
bottom_25 = list(airport_pivot[53:65].index)
bottom_5 = list(airport_pivot[65:])
airport_pivot

,Arrival_15
DESTINATION_AIRPORT,
Trenton Mercer Airport,0.568182
Boise Airport (Boise Air Terminal),0.476190
John Wayne Airport (Orange County Airport),0.322581
Spokane International Airport,0.306452
LaGuardia Airport (Marine Air Terminal),0.284884
Seattle-Tacoma International Airport,0.279803
Greater Rochester International Airport,0.276892
Gen. Edward Lawrence Logan International Airport,0.275298
Newark Liberty International Airport,0.274985


In [36]:
midway_model['Top_5'] = midway_model['DESTINATION_AIRPORT'].apply(lambda x: 1 if x in top_5 else 0)
midway_model['Top_25'] = midway_model['DESTINATION_AIRPORT'].apply(lambda x: 1 if x in top_25 else 0)
midway_model['Second_25'] = midway_model['DESTINATION_AIRPORT'].apply(lambda x: 1 if x in second_25 else 0)
midway_model['Third_25'] = midway_model['DESTINATION_AIRPORT'].apply(lambda x: 1 if x in third_25 else 0)
midway_model['Bottom_25'] = midway_model['DESTINATION_AIRPORT'].apply(lambda x: 1 if x in bottom_25 else 0)
midway_model['Bottom_5'] = midway_model['DESTINATION_AIRPORT'].apply(lambda x: 1 if x in bottom_5 else 0)
midway_model = pd.get_dummies(data=midway_model, columns=['AIRLINE'], drop_first=False)
midway_model.head()

,DAY_OF_WEEK,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIR_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,precip_0hr_before,precip_1hr_before,...,Top_25,Second_25,Third_25,Bottom_25,Bottom_5,AIRLINE_Atlantic Southeast Airlines,AIRLINE_Delta Air Lines Inc.,AIRLINE_Frontier Airlines Inc.,AIRLINE_Skywest Airlines Inc.,AIRLINE_Southwest Airlines Co.
0,4,Seattle-Tacoma International Airport,855,-1.0,251.0,1733,1135,-13.0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,4,Seattle-Tacoma International Airport,1925,23.0,246.0,1733,2200,18.0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,5,Seattle-Tacoma International Airport,855,6.0,244.0,1733,1135,-5.0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,5,Seattle-Tacoma International Airport,1925,22.0,265.0,1733,2200,30.0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,6,Seattle-Tacoma International Airport,850,37.0,246.0,1733,1130,23.0,2,3,...,1,0,0,0,0,0,0,0,0,1


In [37]:
X_features = midway_model.drop(['ARRIVAL_DELAY', 'Arrival_15','DESTINATION_AIRPORT'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_features,midway_model['Arrival_15'],test_size=0.3,random_state=5)

models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RFC', RandomForestClassifier()))
models.append(('BAG', BaggingClassifier()))
models.append(('ADA', AdaBoostClassifier()))


for name, model in models:
    model.fit(X_train, y_train)
    # predict on X_holdout
    yhat_ho = model.predict(X_test)
    print(name, "Holdout Results")
    print(round((accuracy_score(y_test, yhat_ho)*100),1),"% Accuracy")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, yhat_ho))
    print(classification_report(y_test, yhat_ho))

LR Holdout Results
93.9 % Accuracy
Confusion Matrix:
[[17954   465]
 [  982  4205]]
             precision    recall  f1-score   support

          0       0.95      0.97      0.96     18419
          1       0.90      0.81      0.85      5187

avg / total       0.94      0.94      0.94     23606

KNN Holdout Results
91.3 % Accuracy
Confusion Matrix:
[[18098   321]
 [ 1737  3450]]
             precision    recall  f1-score   support

          0       0.91      0.98      0.95     18419
          1       0.91      0.67      0.77      5187

avg / total       0.91      0.91      0.91     23606

CART Holdout Results
90.7 % Accuracy
Confusion Matrix:
[[17243  1176]
 [ 1008  4179]]
             precision    recall  f1-score   support

          0       0.94      0.94      0.94     18419
          1       0.78      0.81      0.79      5187

avg / total       0.91      0.91      0.91     23606

RFC Holdout Results
93.1 % Accuracy
Confusion Matrix:
[[18045   374]
 [ 1247  3940]]
             pr

In [0]:
#midway_model['Distance_bin'] = midway_model['DISTANCE'].apply(lambda x: round ((x/100)))
#midway_model['SD_bin'] = midway_model['SCHEDULED_DEPARTURE'].apply(lambda x: round ((x/100)))
#midway_model['SA_bin'] = midway_model['SCHEDULED_ARRIVAL'].apply(lambda x: 1 if x > 1500 else 0)
#midway_model['Air_time_bin'] = midway_model['AIR_TIME'].apply(lambda x: round ((x/10)))
# midway_model['0h_0precip_bin'] = midway_model['precip_0hr_before'].apply(lambda x: 1 if x == 0 else 0)
# midway_model['0h_1-15precip_bin'] = midway_model['precip_0hr_before'].apply(lambda x: 1 if x>0 & x<16 else 0)
# midway_model['0h_16precip_bin'] = midway_model['precip_0hr_before'].apply(lambda x: 1 if x>=16 else 0)
#midway_model.head()

In [40]:
midway_model.shape

(78685, 25)

In [41]:
midway_model[midway_model['SCHEDULED_DEPARTURE'] != 0].shape

(78685, 25)

In [42]:
for i in range(40):
    print(i,midway_model[midway_model['precip_0hr_before'] >= i]['Arrival_15'].mean())

0 0.21952087437249793
1 0.40071296540797463
2 0.46405579399141633
3 0.4587447108603667
4 0.4796849087893864
5 0.5147900763358778
6 0.5214899713467048
7 0.5315487571701721
8 0.5345454545454545
9 0.5492513790386131
10 0.5358851674641149
11 0.5697297297297297
12 0.582716049382716
13 0.5816733067729084
14 0.5756302521008403
15 0.6
16 0.603202846975089
17 0.6228448275862069
18 0.6431818181818182
19 0.6674698795180722
20 0.6632911392405063
21 0.6766304347826086
22 0.6853932584269663
23 0.6853932584269663
24 0.7040498442367601
25 0.7021943573667712
26 0.7189542483660131
27 0.7152317880794702
28 0.7137809187279152
29 0.706959706959707
30 0.75
31 0.7602040816326531
32 0.7602040816326531
33 0.7602040816326531
34 0.7759562841530054
35 0.7747252747252747
36 0.7241379310344828
37 0.7241379310344828
38 0.7121212121212122
39 0.7121212121212122


In [43]:
for i in range(100,2400, 100):
    print(i,midway_model[midway_model['SCHEDULED_ARRIVAL'] >=i]['Arrival_15'].mean())

100 0.2152467578035384
200 0.2147078758302833
300 0.2147078758302833
400 0.2147078758302833
500 0.2147078758302833
600 0.2147078758302833
700 0.21498017489544294
800 0.21726174052204625
900 0.22439475891591704
1000 0.23244332338736964
1100 0.2420679577405647
1200 0.25336314514626423
1300 0.2628031801121918
1400 0.26953428201811125
1500 0.27605266279207685
1600 0.28413527487601564
1700 0.2962536391848226
1800 0.3019827522665291
1900 0.30901552575864505
2000 0.31162889669113736
2100 0.3096970957232966
2200 0.3014916223947691
2300 0.2963132585298832


In [44]:
#in write up, talk about 3 unsuccessful feature engineering techinques -- grid search, recursive feature elimination,
#and binning (aka quantization)
for i in range(1,13):
    print(i,midway_model[midway_model['Month'] >=i]['Arrival_15'].mean())

1 0.21952087437249793
2 0.21794001523862297
3 0.21789848218709185
4 0.2205397402247796
5 0.22691657704716547
6 0.22893267872755396
7 0.21198406460879698
8 0.19042798296446276
9 0.18366267609019665
10 0.19942217942069784
11 0.19942217942069784
12 0.24369249035322055


In [47]:
X_features = midway_model.drop(['ARRIVAL_DELAY', 'Arrival_15','DESTINATION_AIRPORT'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_features,midway_model['Arrival_15'],test_size=0.3,random_state=5)

clf1 = LogisticRegression()
clf2 = KNeighborsClassifier()
clf3 = BaggingClassifier()


eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bag', clf3)], weights = [6,3,4],
                         voting='soft', flatten_transform = True)
eclf1 = eclf1.fit(X_train, y_train)
print(eclf1.score(X_test, y_test))

0.9403117851393713


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



In [48]:
midway_model.columns

Index(['DAY_OF_WEEK', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE',
       'DEPARTURE_DELAY', 'AIR_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL',
       'ARRIVAL_DELAY', 'precip_0hr_before', 'precip_1hr_before', 'daily_sum',
       'Month', 'Day', 'Arrival_15', 'Top_5', 'Top_25', 'Second_25',
       'Third_25', 'Bottom_25', 'Bottom_5',
       'AIRLINE_Atlantic Southeast Airlines', 'AIRLINE_Delta Air Lines Inc.',
       'AIRLINE_Frontier Airlines Inc.', 'AIRLINE_Skywest Airlines Inc.',
       'AIRLINE_Southwest Airlines Co.'],
      dtype='object')